In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

/Users/siddarthsrinivas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/siddarthsrinivas/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [24]:
tokens = tokenizer.encode('Average product. Great and amazing', return_tensors='pt')
tokens

tensor([[  101, 11237, 20058,   119, 11838, 10110, 39854,   102]])

In [25]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.6242, -0.7069,  0.8921,  0.7754,  0.5908]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
int(torch.argmax(result.logits))+1


3

In [27]:
result.logits

tensor([[-1.6242, -0.7069,  0.8921,  0.7754,  0.5908]],
       grad_fn=<AddmmBackward0>)

In [41]:
r = requests.get('https://www.yelp.com/biz/1796-room-bedford?osq=Restaurants')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [42]:
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [43]:
reviews

["Stopped here for dinner. We had tried eating here the week before and were unable to get a table, so this week I made sure to get the reservation earlier. It wasn't too crowded on a Saturday night, and we were sat immediately with the reservation. We placed an order for drinks to start - I got the regiment drink with bourbon. It was good - a decent mix of hooch and ingredients. For the starters we charged the onion soup and a Caesar salad. The Caesar salad wasn't out of the ordinary and the soup was very good. The broth had an interesting flavor to it. For the entree I went with the 14oz NY strip steak. I added bleu cheese crust. For the temp we were nudged towards a medium plus which is a little more towards well than a regular medium. Well - for the second straight weekend at Bedford Springs I got an undercooked steak. This one I didn't sent back and just drenched it in horseradish sauce. We shared  chocolate cake for dessert which they put a candle on for the celebrated birthday. 

In [44]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [45]:
df['review'].iloc[0]

"Stopped here for dinner. We had tried eating here the week before and were unable to get a table, so this week I made sure to get the reservation earlier. It wasn't too crowded on a Saturday night, and we were sat immediately with the reservation. We placed an order for drinks to start - I got the regiment drink with bourbon. It was good - a decent mix of hooch and ingredients. For the starters we charged the onion soup and a Caesar salad. The Caesar salad wasn't out of the ordinary and the soup was very good. The broth had an interesting flavor to it. For the entree I went with the 14oz NY strip steak. I added bleu cheese crust. For the temp we were nudged towards a medium plus which is a little more towards well than a regular medium. Well - for the second straight weekend at Bedford Springs I got an undercooked steak. This one I didn't sent back and just drenched it in horseradish sauce. We shared  chocolate cake for dessert which they put a candle on for the celebrated birthday. O

In [49]:
df.tail

<bound method NDFrame.tail of                                               review
0  Stopped here for dinner. We had tried eating h...
1  Historic setting w/ unique seating inside Omni...
2  This is a typical steak house.  Drinks are mad...
3  My recent birthday celebration at 1796 Restaur...
4  I really wanted to love it - the service was g...
5  I've driven past the Bedford Omni before but  ...
6  We've really enjoyed this place when visited. ...
7  I have been here a few times over the years an...
8  A fabulous steakhouse!!  Hidden in Bedford, Pa...
9  Very unique steakhouse. The set up and decor t...>

In [47]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [51]:
sentiment_score(df['review'].iloc[3])

5

In [52]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [53]:
df

,review,sentiment
0,Stopped here for dinner. We had tried eating h...,3
1,Historic setting w/ unique seating inside Omni...,3
2,This is a typical steak house. Drinks are mad...,3
3,My recent birthday celebration at 1796 Restaur...,5
4,I really wanted to love it - the service was g...,2
5,I've driven past the Bedford Omni before but ...,5
6,We've really enjoyed this place when visited. ...,3
7,I have been here a few times over the years an...,2
8,"A fabulous steakhouse!! Hidden in Bedford, Pa...",5
9,Very unique steakhouse. The set up and decor t...,5


In [56]:
average = df['sentiment'].mean()
print(average)

3.6
